In [96]:
import pandas as pd 
import re
import seaborn as sns 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.preprocessing import StandardScaler

In [141]:
data_train=pd.read_csv('./Data/train.csv')
data_test=pd.read_csv('./Data/test.csv')
ids_test=data_test.Id

In [142]:
data_test=data_test.drop(['Id'],axis=1)

In [143]:
X=data_train.copy()

In [144]:
#X=X.drop('wheel',axis=1)

In [145]:
#Procesamiento
# Todo a km
X['running']=X['running'].apply(lambda x: float(x.split(' ')[0]) * 1.60934 if x.split(' ')[2]=='miles' else float(x.split(' ')[0]))
data_test['running']=data_test['running'].apply(lambda x: float(x.split(' ')[0]) * 1.60934 if x.split(' ')[2]=='miles' else float(x.split(' ')[0]))

#Sustituir status por la mediana de su categoria
meds={s :X[X['status']==s]['price'].median() for s in X['status'].unique()}

X['status']=X['status'].map(meds)
data_test['status']=data_test['status'].map(meds)

#X['running_per_motor']=X['running'] / X['motor_volume']
#data_test['running_per_motor']=data_test['running'] / data_test['motor_volume']

#X['running_per_year']=X['running'] / (2024-X['year'])
#data_test['running_per_year']=data_test['running'] / (2024-data_test['year'])
#X.loc[:, 'price'] = np.log1p(X.price)
#X.loc[:, 'running'] = np.log1p(X.running)

y=X['price']
X=X.drop('price',axis=1)


In [146]:

num_cols=X.select_dtypes(include='number').columns
scaler=StandardScaler()
X[num_cols]=scaler.fit_transform(X[num_cols])
data_test[num_cols]=scaler.transform(data_test[num_cols])


In [147]:
X=pd.get_dummies(X.drop('color',axis=1)).astype(float)
data_test=pd.get_dummies(data_test.drop('color',axis=1)).astype(float)



X, data_test = X.align(data_test, join='left', axis=1)
data_test = pd.get_dummies(data_test)
X, X_tdata_testest = X.align(data_test, join='left', axis=1)



In [148]:
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, test_size=0.2,random_state=40)

In [149]:
my_model_1 = CatBoostRegressor(loss_function='MAE',random_state=0,silent=True)

# Fit the model

my_model_1.fit(X_train_full, y_train)

predictions_1 = my_model_1.predict(X_valid_full)

# Calculate MAE
mae_1 = mean_absolute_error(predictions_1, y_valid)

#Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)
mean_absolute_error(my_model_1.predict(X_train_full), y_train)

Mean Absolute Error: 1979.3097906898988


1385.2678995508181

In [150]:
#Prueba 2- Fine Tunning
param_grid_gb = {
    'iterations': [50, 100, 200],
    'learning_rate': [0.01,0.05, 0.1],
    'depth': [4, 6, 8],
    'l2_leaf_reg': [ 3, 5, 7],
}

grid_gb = RandomizedSearchCV(
    estimator = CatBoostRegressor(loss_function='MAE', silent = True,random_state=0) ,
    param_distributions = param_grid_gb,
    scoring = 'neg_mean_absolute_error',
    n_jobs = 4,
    verbose=True, 
    cv = 10,
     )
grid_gb.fit(X_train_full,y_train)
print(grid_gb.best_params_)


Fitting 10 folds for each of 10 candidates, totalling 100 fits
{'learning_rate': 0.05, 'l2_leaf_reg': 7, 'iterations': 200, 'depth': 8}


In [151]:
# Calculate MAE
mae_1 = mean_absolute_error(grid_gb.predict(X_valid_full), y_valid)

#Uncomment to print MAE
print("Mean Absolute Error:" , mae_1)

mae_1 = mean_absolute_error(grid_gb.predict(X_train_full), y_train)

#Uncomment to print MAE
print("Train Error:" , mae_1)

Mean Absolute Error: 1948.5965990498153
Train Error: 1503.332197420045


In [152]:
preds=grid_gb.predict(data_test)
output = pd.DataFrame({'Id': ids_test,
                        'Predicted_Price': preds})
output.to_csv('submission.csv', index=False)

In [132]:
#Resultados obtenidos
mod=['Catboost sin color usando medianas para sustituir status','Como el anterior con HY.TUN','FE KM/YEAR','status en dummy','haciendo medianas con type']
train=[1385.27,1947,1944,1930]
test=[1979.31,1932,1927,1983]
externa=[1834.16,1812.2,1861]

data=pd.DataFrame(zip(mod,train,test,externa),columns=['Modelo','MAE TRAIN','MAE TEST','MAE PUBLICO'])
data.head()

,Modelo,MAE TRAIN,MAE TEST,MAE PUBLICO
0,Catboost sin color usando medianas para sustit...,1385.27,1979.31,1834.16
1,Como el anterior con HY.TUN,1947.00,1932.00,1812.20
2,FE KM/YEAR,1944.00,1927.00,1861.00
